# Data Transformers

Keras support many types of input and output data formats, including:

* Multiple inputs
* Multiple outputs
* Higher-dimensional tensors
* Ragged datasets (variable datapoints per observation)
* `tf.data.Dataset`

In this notebook, we explore how to reconcile this functionality with the sklearn ecosystem via SciKeras Data Transformer interface.



<table align="left"><td>
<a target="_blank" href="https://colab.research.google.com/github/adriangb/scikeras/blob/master/notebooks/Basic_Usage.ipyn">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>  
</td><td>
<a target="_blank" href="https://github.com/adriangb/scikeras/blob/master/notebooks/Basic_Usage.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a></td></table>

### Table of contents

* [Data transformer interface](#0)
    * [get_metadata method](#0-1)
* [1. Multiple outputs](#1)
    * [1.1 Define Keras Model](#1-1)
    * [1.2 Define output transformer](1-2)
    * [1.3 Test classifier](#1-3)
* [2. Multiple inputs](#2)
    * [2.1 Define Keras Model](#2-1)
    * [2.2 Define input transformer](#2-2)
    * [2.3 Test regressor](#2-3)
* [3. Multidimensional inputs with MNIST dataset](#3)
    * [3.1 Define Keras Model](#3-1)
    * [3.2 Define transformer](#3-2)
    * [3.3 Test classifier](#3-3-)
* [4. Ragged datasets with tf.data.Dataset](#4)
* [5. Multi-output class_weight](#5)

Install SciKeras

In [ ]:
!python -m pip install scikeras

Silence TensorFlow warnings to keep output succint.

In [138]:
import warnings
from tensorflow import get_logger
get_logger().setLevel('ERROR')
warnings.filterwarnings("ignore", message="Setting the random state for TF")

In [139]:
import numpy as np
from scikeras.wrappers import KerasClassifier, KerasRegressor
from tensorflow import keras

## <a name="0"></a> Data transformer interface

SciKeras enables advanced Keras use cases by providing an interface to convert sklearn compliant data to whatever format your Keras model requires within SciKeras, right before passing said data to the Keras model.

This interface is implemented in the form of two sklearn transformers, one for the features (`X`) and one for the target (`y`).  SciKeras loads these transformers via the `target_encoder` and `feature_encoder` methods.

By default, SciKeras implements `target_encoder` for both KerasClassifier and KerasRegressor to facilitate common types of tasks in sklearn. The default implementations are `scikeras.utils.transformers.ClassifierLabelEncoder` and `scikeras.utils.transformers.RegressorTargetEncoder` for KerasClassifier and KerasRegressor respectively. Information on the types of tasks that these default transformers are able to perform can be found in the [SciKeras docs](https://scikeras.readthedocs.io/en/latest/advanced.html#data-transformers).

Below is an outline of the inner workings of the data transfomer interfaces to help understand when they are called:

In [140]:
if False:  # avoid executing pseudocode
    from scikeras.utils.transformers import (
        ClassifierLabelEncoder,
        RegressorTargetEncoder,
    )


    class BaseWrapper:
        def fit(self, X, y):
            self.target_encoder_ = self.target_encoder
            self.feature_encoder_ = self.feature_encoder
            y = self.target_encoder_.fit_transform(y)
            X = self.feature_encoder_.fit_transform(X)
            self.model_.fit(X, y)
            return self
        
        def predict(self, X):
            X = self.feature_encoder_.transform(X)
            y_pred = self.model_.predict(X)
            return self.target_encoder_.inverse_transform(y_pred)

    class KerasClassifier(BaseWrapper):

        @property
        def target_encoder(self):
            return ClassifierLabelEncoder(loss=self.loss)
        
        def predict_proba(self, X):
            X = self.feature_encoder_.transform(X)
            y_pred = self.model_.predict(X)
            return self.target_encoder_.inverse_transform(y_pred, return_proba=True)


    class KerasRegressor(BaseWrapper):

        @property
        def target_encoder(self):
            return RegressorTargetEncoder()

To substitute your own data transformation routine, you must subclass the wrappers and override one of the encoder defining functions. You will have access to all attributes of the wrappers, and you can pass these to your transformer, like we do above with `loss`.

In [141]:
from sklearn.base import BaseEstimator, TransformerMixin

In [142]:
if False:  # avoid executing pseudocode

    class MultiOutputTransformer(BaseEstimator, TransformerMixin):
        ...


    class MultiOutputClassifier(KerasClassifier):

        @property
        def target_encoder(self):
            return MultiOutputTransformer(...)

### <a name="0-1"></a> get_metadata method

SciKeras recognized an optional `get_metadata` on the transformers. `get_metadata` is expected to return a dicionary of with key strings and arbitrary values. SciKeras will set add these items to the wrappers namespace and make them available to your model building function via the `meta` keyword argument:

In [143]:
if False:  # avoid executing pseudocode

    class MultiOutputTransformer(BaseEstimator, TransformerMixin):
        def get_metadata(self):
            return {"my_param_": "foobarbaz"}


    class MultiOutputClassifier(KerasClassifier):

        @property
        def target_encoder(self):
            return MultiOutputTransformer(...)


    def get_model(meta):
        print(f"Got: {meta['my_param_']}")


    clf = MultiOutputClassifier(model=get_model)
    clf.fit(X, y)  # Got: foobarbaz
    print(clf.my_param_)  # foobarbaz

## <a name="1"></a> 1. Multiple Outputs

Keras makes it striaghtforward to define models with multiple outputs, that is a Model with multiple sets of fully-connected heads at the end of the network. This functionality is only available in the Functional Model and subclassed Model definition modes, and is not available when using Sequential.

In practice, the main thing about Keras models with multiple outputs that you need to know as a SciKeras user is that Keras expects `X` or `y` to be a list of arrays/tensors, with one array/tensor for each input/output.

Note that "multiple outputs" in Keras has a slightly different meaning than "multiple outputs" in sklearn. Many tasks that would be considered "multiple output" tasks in sklearn can be mapped to a single "output" in Keras with multiple units. This notebook specifically focuses on the cases that require multiple distinct Keras outputs.

### <a name="1-1"></a> 1.1 Define Keras Model

Here we define a simple perceptron that has two outputs, corresponding to one binary classification taks and one multiclass classification task. For example, one output might be "image has car" (binary) and the other might be "color of car in image" (multiclass).

In [144]:
def get_clf_model(meta, compile_kwargs):
    inp = keras.layers.Input(shape=(meta["n_features_in_"]))
    x1 = keras.layers.Dense(100, activation="relu")(inp)
    out_bin = keras.layers.Dense(1, activation="sigmoid")(x1)
    out_cat = keras.layers.Dense(meta["n_classes_"][1], activation="softmax")(x1)
    model = keras.Model(inputs=inp, outputs=[out_bin, out_cat])
    model.compile(
        loss=["binary_crossentropy", "sparse_categorical_crossentropy"],
        optimizer=compile_kwargs["optimizer"]
    )
    return model

Let's test that this model works with the kind of inputs and outputs we expect.

In [145]:
X = np.random.random(size=(100, 10))
y_bin = np.random.randint(0, 2, size=(100,))
y_cat = np.random.randint(0, 5, size=(100, ))
y = [y_bin, y_cat]

# build mock meta
meta = {
    "n_features_in_": 10,
    "n_classes_": [2, 5]  # note that we made this a list, one for each output
}
# build mock compile_kwargs
compile_kwargs = {"optimizer": "sgd"}

model = get_clf_model(meta=meta, compile_kwargs=compile_kwargs)

model.fit(X, y, verbose=0)
y_pred = model.predict(X)

In [146]:
print(y_pred[0][:2, :])

[[0.4953647]
 [0.5277547]]


In [147]:
print(y_pred[1][:2, :])

[[0.14030135 0.24718134 0.21445115 0.18598711 0.21207905]
 [0.15261228 0.24328376 0.22087498 0.17868358 0.20454536]]


As you can see, our `predict` output is also a list of arrays, except it contains probabilities instead of the class predictions.

Our data transormer's job will be to convert from a single numpy array (which is what the sklearn ecosystem works with) to the list of arrays and then back. Additionally, for classifiers, we will want to be able to convert probabilities to class predictions.

We will structure our data on the sklearn side by column-stacking our list
of arrays. This works well in this case since we have the same number of datapoints in each array.

### <a name="1-2"></a> 1.2 Define output Data Transformer

Let's go ahead and protoype this data transformer:

In [148]:
from typing import List

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder


class MultiOutputTransformer(BaseEstimator, TransformerMixin):

    def fit(self, y):
        y_bin, y_cat = y[:, 0], y[:, 1]
        # Create internal encoders to ensure labels are 0, 1, 2...
        self.bin_encoder_ = LabelEncoder()
        self.cat_encoder_ = LabelEncoder()
        # Fit them to the input data
        self.bin_encoder_.fit(y_bin)
        self.cat_encoder_.fit(y_cat)
        # Save the number of classes
        self.n_classes_ = [
            self.bin_encoder_.classes_.size,
            self.cat_encoder_.classes_.size,
        ]
        # Save number of expected outputs in the Keras model
        # SciKeras will automatically use this to do error-checking
        self.n_outputs_expected_ = 2
        return self

    def transform(self, y: np.ndarray) -> List[np.ndarray]:
        y_bin, y_cat = y[:, 0], y[:, 1]
        # Apply transformers to input array
        y_bin = self.bin_encoder_.transform(y_bin)
        y_cat = self.cat_encoder_.transform(y_cat)
        # Split the data into a list
        return [y_bin, y_cat]

    def inverse_transform(self, y: List[np.ndarray], return_proba: bool = False) -> np.ndarray:
        y_pred_proba = y  # rename for clarity, what Keras gives us are probs
        if return_proba:
            return np.column_stack(y_pred_proba, axis=1)
        # Get class predictions from probabilities
        y_pred_bin = (y_pred_proba[0] > 0.5).astype(int).reshape(-1, )
        y_pred_cat = np.argmax(y_pred_proba[1], axis=1)
        # Pass back through LabelEncoder
        y_pred_bin = self.bin_encoder_.inverse_transform(y_pred_bin)
        y_pred_cat = self.cat_encoder_.inverse_transform(y_pred_cat)
        return np.column_stack([y_pred_bin, y_pred_cat])
    
    def get_metadata(self):
        return {
            "n_classes_": self.n_classes_,
            "n_outputs_expected_": self.n_outputs_expected_,
        }

Note that in addition to the usual `transform` and `inverse_transform` methods, we implement the `get_metadata` method to return the `n_classes_` attribute.

Lets test our transformer with the same dataset we previoulsy used to test our model:

In [149]:
tf = MultiOutputTransformer()

y_sklearn = np.column_stack(y)

y_keras = tf.fit_transform(y_sklearn)
print("`y`, as will be passed to Keras:")
print([y_keras[0][:4], y_keras[1][:4]])

`y`, as will be passed to Keras:
[array([1, 0, 1, 0]), array([4, 4, 3, 2])]


In [150]:
y_pred_sklearn = tf.inverse_transform(y_pred)
print("`y_pred`, as will be returned to sklearn:")
y_pred_sklearn[:5]

`y_pred`, as will be returned to sklearn:


array([[0, 1],
       [1, 1],
       [0, 4],
       [0, 4],
       [0, 4]])

In [151]:
print(f"metadata = {tf.get_metadata()}")

metadata = {'n_classes_': [2, 5], 'n_outputs_expected_': 2}


Since this looks good, we move on to integrating our transformer into our classifier.

In [152]:
class MultiOutputClassifier(KerasClassifier):

    @property
    def target_encoder(self):
        return MultiOutputTransformer()

### <a name="1-3"></a> 1.3 Test classifier

In [153]:
from sklearn.preprocessing import StandardScaler

# First we build an artifical dataset where the features are highly correlated with the labels
X = y_sklearn
X = StandardScaler().fit_transform(X)

In [154]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

# We need a custom scorer for this dataset
# See https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
def scorer(estimator, X, y):
    y_pred = estimator.predict(X)
    y_bin, y_cat = y[:, 0], y[:, 1]
    y_pred_bin, y_pred_cat = y_pred[:, 0], y_pred[:, 1]
    return np.mean([accuracy_score(y_bin, y_pred_bin), accuracy_score(y_cat, y_pred_cat)])

In [155]:
clf = MultiOutputClassifier(model=get_clf_model, verbose=0, random_state=0, epochs=100,)

np.mean(cross_val_score(clf, X, y_sklearn, scoring=scorer))

0.985

## <a name="2"></a> 2. Multiple inputs

The process for multiple inputs is similar, but instead of overriding the transformer in `target_encoder` we override `feature_encoder`.

In [156]:
if False:  # avoid running pseudocode
    from sklearn.base import BaseEstimator, TransformerMixin


    class MultiInptuTransformer(BaseEstimator, TransformerMixin):
        ...


    class MultiInputClassifier(KerasClassifier):

        @property
        def feature_encoder(self):
            return MultiInputTransformer(...)

### <a name="1-1"></a>2.1 Define Keras Model

Let's define a Keras **regression** Model with 2 inputs:

In [157]:
def get_reg_model(compile_kwargs):

    inp1 = keras.layers.Input(shape=(1, ))
    inp2 = keras.layers.Input(shape=(1, ))

    x1 = keras.layers.Dense(100, activation="relu")(inp1)
    x2 = keras.layers.Dense(50, activation="relu")(inp2)

    concat = keras.layers.Concatenate(axis=-1)([x1, x2])

    out = keras.layers.Dense(1)(concat)

    model = keras.Model(inputs=[inp1, inp2], outputs=out)
    model.compile(loss="mse", optimizer=compile_kwargs["optimizer"])

    return model

And test it with a small mock dataset:

In [158]:
X = np.random.random(size=(100, 2))
y = np.sum(X, axis=1)
X = np.split(X, 2, axis=1)

# build mock compile_kwargs
compile_kwargs = {"optimizer": "sgd"}

model = get_reg_model(compile_kwargs=compile_kwargs)

model.fit(X, y, verbose=0, epochs=100)
y_pred = model.predict(X).squeeze()

In [159]:
from sklearn.metrics import r2_score

r2_score(y, y_pred)

0.8902906544625109

Having verified that our model builds without errors and accepts the inputs types we expect, we move onto integrating a transformer into our SciKeras model.

### 2.2 Define Data Transformer

Just like for overriding `target_encoder`, we just need to define a sklearn transformer and drop it into our SciKeras wrapper. Since we hardcoded the input
shapes into our model and do not rely on any transformer-generated metadata, we can simply use `sklearn.preprocessing.FunctionTransformer`:

In [160]:
from sklearn.preprocessing import FunctionTransformer


class MultiInputRegressor(KerasRegressor):

    @property
    def feature_encoder(self):
        return FunctionTransformer(
            func=lambda X: [X[:, 0], X[:, 1]],
        )

Note that we did **not** implement `inverse_transform` (that is, we did not pass an `inverse_func` argument to `FunctionTransformer`) because features are never converted back to their original form.

### 2.3 Test regressor

In [161]:
reg = MultiInputRegressor(model=get_reg_model, epochs=100, verbose=0, random_state=0)

X_sklearn = np.column_stack(X)

np.mean(cross_val_score(reg, X_sklearn, y))

0.9995720499457423

## 3. Multidimensional inputs with MNIST dataset

In this example, we look at how we can use SciKeras to process the MNIST dataset. The dataset is composed of 60,000 images of digits, each of which is a 2D 28x28 image.

The dataset and Keras Model architecture used come from a [Keras example](https://keras.io/examples/vision/mnist_convnet/). It may be beneficial to understand the Keras model by reviewing that example first.

In [162]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train.shape

(60000, 28, 28)

The outputs (labels) are numbers 0-9:

In [163]:
print(y_train.shape)
print(np.unique(y_train))

(60000,)
[0 1 2 3 4 5 6 7 8 9]


First, we will "flatten" the data into an array of shape `(n_samples, 28*28)` (i.e. a 2D array). This will allow us to use sklearn ecosystem utilities, for example, `sklearn.preprocessing.MinMaxScaler`.

In [164]:
from sklearn.preprocessing import MinMaxScaler

n_samples_train = x_train.shape[0]
n_samples_test = x_test.shape[0]

x_train = x_train.reshape((n_samples_train, -1))
x_test = x_test.reshape((n_samples_test, -1))
x_train = MinMaxScaler().fit_transform(x_train)
x_test = MinMaxScaler().fit_transform(x_test)

In [165]:
print(x_train.shape[1:])  # 784 = 28*28

(784,)


In [166]:
print(np.min(x_train), np.max(x_train))  # scaled 0-1

0.0 1.0


Of course, in this case, we could have just as easily used numpy functions to scale our data, but we use `MinMaxScaler` to demonstrate use of the sklearn ecosystem.

### 3.1 Define Keras Model

Next we will define our Keras model (adapted from [keras.io](https://keras.io/examples/vision/mnist_convnet/)):

In [167]:
num_classes = 10
input_shape = (28, 28, 1)


def get_model(meta):
    model = keras.Sequential(
        [
            keras.Input(input_shape),
            keras.layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),
            keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            keras.layers.MaxPooling2D(pool_size=(2, 2)),
            keras.layers.Flatten(),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(num_classes, activation="softmax"),
        ]
    )
    model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam"
    )
    return model

Now let's define a transformer that we will use to reshape our input from the sklearn shape (`(n_samples, 784)`) to the Keras shape (which we will be `(n_samples, 28, 28, 1)`).

In [168]:
class MultiDimensionalClassifier(KerasClassifier):

    @property
    def feature_encoder(self):
        return FunctionTransformer(
            func=lambda X: X.reshape(X.shape[0], *input_shape),
        )

In [169]:
clf = MultiDimensionalClassifier(
    model=get_model,
    epochs=5,
    batch_size=128,
    validation_split=0.1,
    random_state=0,
)

### 3.2 Test

Train and score the model (this takes some time)

In [170]:
clf.fit(x_train, y_train)

Epoch 1/15
422/422 [==============================] - 17s 40ms/step - loss: 0.7640 - val_loss: 0.0871
Epoch 2/15
422/422 [==============================] - 18s 42ms/step - loss: 0.1219 - val_loss: 0.0600
Epoch 3/15
422/422 [==============================] - 17s 41ms/step - loss: 0.0863 - val_loss: 0.0454
Epoch 4/15
422/422 [==============================] - 20s 47ms/step - loss: 0.0755 - val_loss: 0.0435
Epoch 5/15
422/422 [==============================] - 19s 46ms/step - loss: 0.0645 - val_loss: 0.0438
Epoch 6/15
422/422 [==============================] - 18s 42ms/step - loss: 0.0553 - val_loss: 0.0361
Epoch 7/15
422/422 [==============================] - 18s 42ms/step - loss: 0.0508 - val_loss: 0.0372
Epoch 8/15
422/422 [==============================] - 18s 42ms/step - loss: 0.0512 - val_loss: 0.0342
Epoch 9/15
422/422 [==============================] - 18s 42ms/step - loss: 0.0450 - val_loss: 0.0307
Epoch 10/15
422/422 [==============================] - 17s 41ms/step - loss: 0.042

MultiDimensionalClassifier(
	model=<function get_model at 0x14dd1b9d0>
	build_fn=None
	warm_start=False
	random_state=0
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=128
	verbose=1
	callbacks=None
	validation_split=0.1
	shuffle=True
	run_eagerly=False
	epochs=15
	class_weight=None
)

In [171]:
score = clf.score(x_test, y_test)
print(f"Test score (accuracy): {score:.2f}")

79/79 [==============================] - 1s 14ms/step
Test score (accuracy): 0.99


## 4. Ragged datasets with tf.data.Dataset

SciKeras provides a third dependancy injection point that operats on the entire dataset: X, y & sample_weight. This `dataset_transformer` is applied after `target_transformer` and `feature_transformer`. One use case for this dependancy injection point is to transform data from tabular/array-like to the `tf.data.Dataset` format, which only requires iteration. We can use this to create a `tf.data.Dataset` of ragged tensors.

Note that `dataset_transformer` should accept a single **3 element tuple** as its argument and return value:

In [172]:
help(KerasClassifier.dataset_transformer)

Help on property:

    Retrieve a transformer to be applied jointly to the entire
    dataset (X, y & sample_weights).
    
    By default, KerasClassifier implements ClassWeightDataTransformer,
    which embeds class_weight into sample_weight.
    
    You can override this method to provide custom transformations.
    To keep the default class_weight behavior, you can chain your
    transfromer and ClassWeightDataTransformer using a Pipeline.
    
    It MUST accept a 3 element tuple as it's single input argument
    to `fit` and `transform`. `transform` must also output
    a 3 element tuple in the same format.
    The first element corresponds to X, or as an output from the
    transformer, to a `tf.data.Dataset` instance containing
    X, y and optionally sample_weights.
    The second element corresponds to `y`, and may be None
    on the output side.
    The third element is `sample_weights` which may be None
    on the input and output sides.
    
    Note that `inverse_transfo

When you return a tuple like `(tf.data.Dataset(...), None, None)`, SciKeras will pass the data untouched to `Model.fit` like `Model.fit(x=tf.data.Dataset(...), y=None, sample_weight=None)`.

Let's start by defining our data. We'll have an extra "feature" that marks the observation index, but we'll remove it when we deconstruct our data in the transformer.

In [173]:
feature_1 = np.random.uniform(size=(10, ))
feature_2 = np.random.uniform(size=(10, ))
obs = [0] * 3 + [1] * 2 + [2] * 1 + [3] * 2 + [4] * 2

X = np.column_stack([feature_1, feature_2, obs]).astype("float32")

y = np.array(["class1"] * 5 + ["class2"] * 5, dtype=str)

Next, we define our `dataset_transformer`. We will do this by defining a custom forward transformation outside of the Keras model. Note that we do not define an inverse transformation since that is never used.
Also note that `dataset_transformer` will _always_ be called with `X` (i.e. the first element of the tuple will always be populated), but will be called with `y=None` when used for `predict`. Thus,
you should check if `y` and `sample_weigh` are None before doing any operations on them.

In [174]:
from typing import Tuple, Optional

from sklearn.base import BaseEstimator, TransformerMixin
from tensorflow import RaggedTensor
from tensorflow.data import Dataset


def dataset_transformer(data: Tuple[np.ndarray, Optional[np.ndarray], Optional[np.ndarray]]) -> Tuple[Dataset, None, None]:
    X, y, sample_weights = data
    if y is not None:
        y = y.reshape(-1, 1 if len(y.shape) == 1 else y.shape[1])
        y = y[RaggedTensor.from_value_rowids(y, X[:, -1]).row_starts().numpy()]
    if sample_weights is not None:
        sample_weights = sample_weights.reshape(-1, 1 if len(sample_weights.shape) == 1 else sample_weights.shape[1])
        sample_weights = sample_weights[RaggedTensor.from_value_rowids(sample_weights, X[:, -1]).row_starts().numpy()]
    X = RaggedTensor.from_value_rowids(X[:, :-1], X[:, -1])
    return (Dataset.from_tensor_slices((X, y, sample_weights)), None, None)


Lets quickly test our transformer:

In [175]:
data = dataset_transformer((X, y, None))
data

(<TensorSliceDataset shapes: ((None, 2), (1,), NoneTensorSpec()), types: (tf.float32, tf.string, NoneTensorSpec())>,
 None,
 None)

In [176]:
data = dataset_transformer((X, None, None))
data

(<TensorSliceDataset shapes: ((None, 2), NoneTensorSpec(), NoneTensorSpec()), types: (tf.float32, NoneTensorSpec(), NoneTensorSpec())>,
 None,
 None)

Our shapes look good, and we can handle the `y=None` case.
Next, we can add our transormer to our model.

In [177]:
from sklearn.preprocessing import FunctionTransformer


class RaggedClassifier(KerasClassifier):

    @property
    def dataset_transformer(self):
        return FunctionTransformer(dataset_transformer)

Now we can define a Model. We need some way to handle/flatten our ragged arrays within our model. For this example, we use a custom mean layer, but you could use an Embedding layer, LSTM, etc.

In [178]:
from tensorflow import reduce_mean, reshape
from tensorflow.keras import Sequential, layers


class CustomMean(layers.Layer):

    def __init__(self, axis=None):
        super(CustomMean, self).__init__()
        self._supports_ragged_inputs = True
        self.axis = axis

    def call(self, inputs, **kwargs):
        input_shape = inputs.get_shape()
        return reshape(reduce_mean(inputs, axis=self.axis), (1, *input_shape[1:]))


def get_model(meta):
    inp_shape = meta["X_shape_"][1]-1
    model = Sequential([               
        layers.Input(shape=(inp_shape,), ragged=True),
        CustomMean(axis=0),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

And attatch our model to our classifier wrapper:

In [179]:
clf = RaggedClassifier(get_model, loss="bce")

Finally, let's train and predict:

In [180]:
clf.fit(X, y)
y_pred = clf.predict(X)
y_pred

5/5 [==============================] - 0s 968us/step


array(['class1', 'class1', 'class1', 'class1', 'class1'], dtype='<U6')

If we define our custom layers, transformers and wrappers in their own module, we can easily create a self-contained classifier that is able to handle ragged datasets and has a clean Scikit-Learn compatible API:

In [181]:
class RaggedClassifier(KerasClassifier):

    @property
    def dataset_transformer(self):
        return FunctionTransformer(dataset_transformer)
    
    def _keras_build_fn(self):
        inp_shape = self.X_shape_[1] - 1
        model = Sequential([               
            layers.Input(shape=(inp_shape,), ragged=True),
            CustomMean(axis=0),
            layers.Dense(1, activation='sigmoid')
        ])
        return model

In [182]:
clf = RaggedClassifier(loss="bce")
clf.fit(X, y)
y_pred = clf.predict(X)
y_pred

5/5 [==============================] - 0s 939us/step


array(['class1', 'class1', 'class1', 'class1', 'class1'], dtype='<U6')

## 5. Multi-output class_weight

In this example, we will use `dataset_transformer` to support multi-output class weights. We will re-use our `MultiOutputTransformer` from our previous example to split the output, then we will create `sample_weights` from `class_weight`

In [183]:
from collections import defaultdict

from sklearn.utils.class_weight import compute_sample_weight


class DatasetTransformer(BaseEstimator, TransformerMixin):

    def __init__(self, output_names, class_weight=None):
        self.class_weight = class_weight
        self.output_names = output_names

    def fit(self, data: Tuple[np.ndarray, Optional[np.ndarray], Optional[np.ndarray]]) -> "DatasetTransformer":
        return self

    def transform(self, data: Tuple[np.ndarray, Optional[np.ndarray], Optional[np.ndarray]]) -> Tuple[Dataset, None, None]:
        if self.class_weight is None:
            return data
        class_weight = self.class_weight
        if isinstance(class_weight, str):  # handle "balanced"
            class_weight_ = class_weight
            class_weight = defaultdict(lambda: class_weight_)
        X, y, sample_weights = data
        assert sample_weights is None, "Cannot use class_weight & sample_weights together"
        if y is not None:
            # y should be a list of arrays, as split up by MultiOutputTransformer
            sample_weights = dict()
            for output_num, (output_name, output_data) in enumerate(zip(self.output_names, y)):
                # class_weight is expected to be indexable by output_number
                # see https://scikit-learn.org/stable/modules/generated/sklearn.utils.class_weight.compute_sample_weight.html
                # Note that it is trivial to change the expected format to match Keras' ({output_name: weights, ...})
                # see https://github.com/keras-team/keras/issues/4735#issuecomment-267473722
                cls_wt_out = class_weight[output_num]
                sample_weights[output_name] = compute_sample_weight(cls_wt_out, output_data)
        return X, y, sample_weights


In [184]:
def get_model(meta, compile_kwargs):
    inp = keras.layers.Input(shape=(meta["n_features_in_"]))
    x1 = keras.layers.Dense(100, activation="relu")(inp)
    out_bin = keras.layers.Dense(1, activation="sigmoid")(x1)
    out_cat = keras.layers.Dense(meta["n_classes_"][1], activation="softmax")(x1)
    model = keras.Model(inputs=inp, outputs=[out_bin, out_cat])
    model.compile(
        loss=["binary_crossentropy", "sparse_categorical_crossentropy"],
        optimizer=compile_kwargs["optimizer"]
    )
    return model


class CustomClassifier(KerasClassifier):

    @property
    def target_encoder(self):
        return MultiOutputTransformer()
    
    @property
    def dataset_transformer(self):
        return DatasetTransformer(
            output_names=self.model_.output_names,
            class_weight=self.class_weight
        )

Next, we define the data. We'll use `sklearn.datasets.make_blobs` to generate a relatively noisy dataset:

In [185]:
from sklearn.datasets import make_blobs


X, y = make_blobs(centers=3, random_state=0, cluster_std=20)
# make a binary target for "is the value of the first class?"
y_bin = y == y[0]
y = np.column_stack([y_bin, y])

Test the model without specifying class weighting:

In [186]:
clf = CustomClassifier(get_model, epochs=100, verbose=0, random_state=0)
clf.fit(X, y)
y_pred = clf.predict(X)
(_, counts_bin) = np.unique(y_pred[:, 0], return_counts=True)
print(counts_bin)
(_, counts_cat) = np.unique(y_pred[:, 1], return_counts=True)
print(counts_cat)

[91  9]
[28 30 42]


As you can see, without `class_weight="balanced"`, our classifier only predicts mainly a single class for the first output. Now with `class_weight="balanced"`:

In [187]:
clf = CustomClassifier(get_model, class_weight="balanced", epochs=100, verbose=0, random_state=0)
clf.fit(X, y)
y_pred = clf.predict(X)
(_, counts_bin) = np.unique(y_pred[:, 0], return_counts=True)
print(counts_bin)
(_, counts_cat) = np.unique(y_pred[:, 1], return_counts=True)
print(counts_cat)

[57 43]
[27 27 46]


Now, we get (mostly) balanced classes. But what if we want to specify our classes manually? You will notice that in when we defined `DatasetTransformer`, we gave it the ability to handle
a list of class weights. For demonstration purposes, we will highly bias towards the second class in each output:

In [188]:
clf = CustomClassifier(get_model, class_weight=[{0: 0.1, 1: 1}, {0: 0.1, 1: 1, 2: 0.1}], epochs=100, verbose=0, random_state=0)
clf.fit(X, y)
y_pred = clf.predict(X)
(_, counts_bin) = np.unique(y_pred[:, 0], return_counts=True)
print(counts_bin)
(_, counts_cat) = np.unique(y_pred[:, 1], return_counts=True)
print(counts_cat)

[ 7 93]
[ 2 98]


Or mixing the two methods, because our first output is unbalanced but our second is (presumably) balanced:

In [189]:
clf = CustomClassifier(get_model, class_weight=["balanced", None], epochs=100, verbose=0, random_state=0)
clf.fit(X, y)
y_pred = clf.predict(X)
(_, counts_bin) = np.unique(y_pred[:, 0], return_counts=True)
print(counts_bin)
(_, counts_cat) = np.unique(y_pred[:, 1], return_counts=True)
print(counts_cat)

[57 43]
[30 25 45]
